In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator
from embeddings_tools import get_embeddings
from intelligence_tools import AnalyzeVariables

In [3]:
df = pd.read_csv('../datasets/Amphibians.csv',sep=';')

In [4]:
df.columns = df.iloc[0].values
df = df.iloc[1:]
df.to_csv('Amphibians_sep.csv', index=False)
df.head()


,ID,Motorway,SR,NR,TR,VR,SUR1,SUR2,SUR3,UR,...,BR,MR,CR,Green frogs,Brown frogs,Common toad,Fire-bellied toad,Tree frog,Common newt,Great crested newt
1,1,A1,600,1,1,4,6,2,10,0,...,0,0,1,0,0,0,0,0,0,0
2,2,A1,700,1,5,1,10,6,10,3,...,1,0,1,0,1,1,0,0,1,0
3,3,A1,200,1,5,1,10,6,10,3,...,1,0,1,0,1,1,0,0,1,0
4,4,A1,300,1,5,0,6,10,2,3,...,0,0,1,0,0,1,0,0,0,0
5,5,A1,600,2,1,4,10,2,6,0,...,5,0,1,0,1,1,1,0,1,1


In [5]:
av = AnalyzeVariables(df)

In [8]:
bdf = BaseDatasetTransform(df,target=['Green frogs','Brown frogs',	'Common toad',	'Fire-bellied toad',	'Tree frog',	'Common newt',	'Great crested newt'])

In [9]:
X_train, X_test, y_train, y_test,categorical_features = bdf.fit_transform()


Обнаружены колонки с именем "id": ['ID']
Колонка "ID" удалена.
Series([], dtype: int64)

В колонке(ах) ['Green frogs', 'Brown frogs', 'Common toad', 'Fire-bellied toad', 'Tree frog', 'Common newt', 'Great crested newt'] нет пропущенных значений
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Motorway', 'SR', 'NR', 'TR', 'VR', 'SUR1', 'SUR2', 'SUR3', 'UR', 'FR',
       'OR', 'RR', 'BR', 'MR', 'CR'],
      dtype='object')

Числовые колонки:
Index([], dtype='object')


In [10]:
from sklearn.multioutput import MultiOutputClassifier


lgbm_model = MultiOutputClassifier(lgb.LGBMClassifier(verbose=-1))

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
)

7


In [11]:
#metrics = [ 'accuracy','f1_samples','precision', 'recall', 'f1_macro','hamming_loss','jaccard_score']
metrics = ['accuracy', 'precision', 'f1_macro','recall']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics,base_multitarget=True)

evaluator.fit(X_train, y_train)

ModelEvaluator(base_multitarget=True,
               metrics=['accuracy', 'precision', 'f1_macro', 'recall'],
               model_trainer=BaseModelTrain(categorical_features=['Motorway',
                                                                  'SR', 'NR',
                                                                  'TR', 'VR',
                                                                  'SUR1',
                                                                  'SUR2',
                                                                  'SUR3', 'UR',
                                                                  'FR', 'OR',
                                                                  'RR', 'BR',
                                                                  'MR', 'CR'],
                                            columns_to_delete=[],
                                            model=MultiOutputClassifier(estimator=LGBMClassifier(verbose=-1))))

In [12]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,accuracy,precision,f1_macro,recall,Targets
0,0.596491,0.715415,0.570586,0.63625,Green frogs
1,0.77193,0.640909,0.50101,0.527098,Brown frogs
2,0.596491,0.298246,0.373626,0.5,Common toad
3,0.736842,0.368421,0.424242,0.5,Fire-bellied toad
4,0.561404,0.565714,0.466891,0.528536,Tree frog
5,0.754386,0.552857,0.538194,0.536561,Common newt
6,0.929825,0.464912,0.481818,0.5,Great crested newt
